# Analyza dat

In [8]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.initializers import Constant

V projekte budeme pracovat s datami MovieLens 1M. Pouzijeme 2 datasety, obsahujuce informacie o filmoch a ich hodnoteniach.

In [2]:
# df_movies = pd.read_csv('../data/raw/movies.dat', sep='::', encoding = "ISO-8859-1", names=['movie_id', 'title', 'category'])
df_ratings = pd.read_csv('../data/raw/ratings.dat', sep='::', encoding = "ISO-8859-1", names=['user_id', 'movie_id', 'rating', 'timestamp'])

/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Dataset filmov obsahuje 3 atributy: id filmu, nazov filmu a kategoriu.

In [ ]:
# df_movies.head()


V datasete sa nachadza 3883 zaznamov, ktore su plnohodnotne a pre nijaky atribut nechybaju udaje.

In [4]:
# df_movies.count()

movie_id    3883
title       3883
category    3883
dtype: int64

Dataset hodnoteni obsahuje 4 atributy: id filmu, id pouzivatela, hodnotenie a cas hodnotenia.

In [14]:
df_ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


V datasete sa nachadza 1000209 zaznamov, ktore su plnohodnotne a pre nijaky atribut nechybaju udaje.

In [7]:
df_ratings.count()


user_id      1000209
movie_id     1000209
rating       1000209
timestamp    1000209
dtype: int64

Cas hodnotenia je vyjadreny ako cislo, preto si ho prekonvertujeme na casovy udaj.

In [3]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'], unit='s')

Pre kazdeho pouzivatela vytvorime zoznam filmov, ktore hodnotil, v poradi, v akom ich hodnotil.

In [4]:
rating_sequences = []
max_sequence = 0
for user in df_ratings['user_id'].unique():
    sequence = df_ratings.loc[df_ratings['user_id'] == user].sort_values(by=['timestamp'])['movie_id']
    rating_sequences.append(sequence.astype(str).tolist())
    if len(sequence) > max_sequence:
        max_sequence = len(sequence)

Zo sekvencii si natrenujeme Word2Vec model.
    

In [5]:
model = Word2Vec(rating_sequences, size=max_sequence, window=5, min_count=5, workers=10, sg=0)

In [6]:
model.wv.save_word2vec_format('../data/movie_vectors.txt')

Vytvorime embedding.

In [7]:
movies_count = len(df_ratings['movie_id'].unique())

In [9]:
embedding_matrix = np.zeros((movies_count, max_sequence))
for i, movie_id in enumerate(df_ratings['movie_id'].unique()):
    item_vector = model.wv[movie_id]
    if item_vector is not None:
        embedding_matrix[i] = item_vector
        

TypeError: 'numpy.int64' object is not iterable

In [ ]:
embedding = Embedding(input_dim=1991, output_dim=300, mask_zero=True, embeddings_initializer=Constant(embedding_matrix))
lstm = Bidirectional(LSTM(64, return_sequences=True))
dense = Dense(units=len(pos_vocab), activation='softmax')

       
#    def call(self, x):
#        x = self.embedding(x)
#        x = self.lstm(x)
#        x = self.dense(x)
#        return x